In [2]:
using Printf
using Plots
using LinearAlgebra # do operacji macierzowych i testowania 

# NOTES:
# identity: Matrix(I, rows, cols)
# random: rand(rows, cols)
# transpose: matrix'
# one row tmp[1:1, :]
# cats: [A B]=hcat [A; B]=vcat
# vector/matrix length/norm = norm([1 2 3]) !!! norma Frobeniusa
# subarray M[2:3, 4:5]
# eltype(M) type of Matrix elements
# NamedTuple{(:a, :b), Tuple{Matrix, Matrix}}((Matrix{Float64}(I, 5, 5), rand(4, 4))).b

Znajdujemy rozkład macieży a na macieże `Q`(ortogonalną) i `R`(górnotrójkątną). Dzieki temu `rank(A)=rank(QR)=rank(R)`, a `rank(R)` jest prostym policzeniem niezerowych wierszy. Korzystam z metody Hausholdera, ze względu na lepszą numeryczną poprawność niż metoda Gramma-Schmidta.

In [29]:
# https://pl.wikipedia.org/wiki/Rozk%C5%82ad_QR
# liczy podmacierz Hausholdera dla wektora `aₙₓ₁` jako Hₙₓₙ=I-2*(v*vᵀ)/(vᵀ*v) gdzie v = a - |a| * [1₁ 0₂ ... 0ₙ]ᵀ, v!=0
# macierz zwracana spełnia warunek H*a=|a|*e, więc dla a=d*e H=I, gdzie d ∈ R, e = [1₁ 0₂ ... 0ₙ]ᵀ
function _HausholderMatrix(a) # O(n²)
    local T, n = eltype(a), length(a)
    if (all(x->x==0, a[2:n])) return Matrix{T}(I, n, n) end # zwraca identyczność gdy a=d*e, bo inaczej v=0
    local v = a+sign(a[1])*norm(a)*[1 zeros(T, n-1)']' # a+sign(a[1])*|a|... zamiast a-|a|... bo utrata cyfr znaczących
    v = v/v[1] # normalizacja ze względu na pierwszy element
    return Matrix{T}(I, n, n) - (2/(v'*v)[1]) * (v*v') # O(n²)
end

# liczy rząd M używając rozkładu QR (Mₙₓₘ=QₙₓₘRₘₓₘ gdzie Q ortogonana i R górnotrójkątna).
# funkcja robi pivoting wybierając wektor o największej normie
function rankQRHausholder(M)
    # R=...H₂H₁M   Q=H₁H₂... (ale Q jest zbędne)
    local T, n, m = eltype(M), length(M[:, 1]), length(M[1, :]) # rows and columns
    if (n < m) n, m, M = m, n, M' end # transponuj. Rząd kolumnowy i wierszowy są takie same a rk(M)=rk(Mᵀ)=rk(Rᵀ)
    local R = convert(Array{T}, Matrix(M))
    for i in 1:min(m, n-1)
#         display(i);
#         display(R)
#         display(R[i:n, i:i]')
        R[:, i:m] = sortslices(R[:, i:m], dims=2, lt=(x,y)->norm(x[i:n])<norm(y[i:n]), rev=true) # pivot
        R = [Matrix{T}(I, i-1, i-1) zeros(T, i-1, n-i+1); zeros(T, n-i+1, i-1) _HausholderMatrix(R[i:n, i:i])]*R
        R[i+1:n, i:i] = zeros(T, n-i)' # naprawmy zera które powinny tu występować aby nie mnożyć błędów numerycznych
    end
    return R # TODO: liczenie rzędu macierzy górnotrójkątnej
end

rankQRHausholder (generic function with 1 method)

In [46]:
A = [12 -51 4; 6 167 -68; -4 24 -41]
tmp = rand(-10:10, 6, 5)
myR, qrR = rankQRHausholder(tmp3), qr(tmp3).R
display(myR)
display(qrR)
norm(myR)-norm(qrR)

6×5 Array{Float64,2}:
 18.4932  -0.162221  14.1673    1.40592  18.4932     
  0.0     13.0757    -1.12436  -8.54807   1.7612e-15 
  0.0      0.0       -8.06364   7.38242  -1.41603e-16
  0.0      0.0        0.0       3.23322   6.95251e-16
  0.0      0.0        0.0       0.0       1.5444e-15 
  0.0      0.0        0.0       0.0       0.0        

5×5 Array{Float64,2}:
 18.4932  18.4932       14.1673   -0.162221   1.40592
  0.0      2.18689e-15  -1.40568  -9.49988    6.92462
  0.0      0.0           8.01939  -3.49846   -5.01089
  0.0      0.0           0.0      -8.27568    7.67542
  0.0      0.0           0.0       0.0       -2.46004

7.105427357601002e-15

Problem jest znalezienie rzędu R. Do tego jest algorytm "Rank Revealing QR decomposition" aka RRQR, gdzie znajdujemy dodatkową macierz P permutacji wejsciowej macierzy A. Dzięki temu wyjściowa macierz R przyjmie postać macierzy górnotrójkątnej z zerowymi wierszami, które będzie można policzyć. Permutacje wykonywane są po kolumnach i posortowane po największej normie wektora.

In [1]:
tmp = rand(-10:10, 6, 5)
display(tmp)


6×5 Array{Int64,2}:
 -10   -6  -6  -9   4
  -2   -5  10  -4   8
   5    4   9   0  -1
  -8  -10   8   1   1
  -7   -9   7   8  -3
  10    3   7  -3   7

Inny sposób: rozkład SVD

In [27]:
tmp2 = sortslices(tmp, dims=2, lt=(x,y)->max(norm(x[3:6]))<max(norm(y[3:6])), rev=true)
display(tmp2)
for i in 1:5
    display(norm(tmp[3:6, i]))
end
tmp

6×5 Array{Int64,2}:
 -6  -10   -6  -9   4
 10   -2   -5  -4   8
  9    5    4   0  -1
  8   -7  -10   1   1
  7   -8   -9   8  -3
  7   10    3  -3   7

15.427248620541512

14.352700094407323

15.588457268119896

8.602325267042627

7.745966692414834

6×5 Array{Int64,2}:
 -10   -6  -6  -9   4
  -2   -5  10  -4   8
   5    4   9   0  -1
  -7  -10   8   1   1
  -8   -9   7   8  -3
  10    3   7  -3   7

In [31]:
tmp3 = tmp2

6×5 Array{Int64,2}:
 -6  -10   -6  -9   4
 10   -2   -5  -4   8
  9    5    4   0  -1
  8   -7  -10   1   1
  7   -8   -9   8  -3
  7   10    3  -3   7

In [45]:
tmp3

6×5 Array{Int64,2}:
 -10  -10   -6  -9   4
  -2   -2   -5  -4   8
   5    5    4   0  -1
  -7   -7  -10   1   1
  -8   -8   -9   8  -3
  10   10    3  -3   7